In [2]:
%pip install pybaseball -q

     |████████████████████████████████| 415 kB 5.0 MB/s 
     |████████████████████████████████| 291 kB 48.7 MB/s 
     |████████████████████████████████| 856 kB 35.7 MB/s 


In [3]:
import pybaseball as pyb
from pybaseball import statcast
from pybaseball import statcast_pitcher, playerid_lookup
from pybaseball import pitching_stats_bref

In [4]:
import pandas as pd
import numpy as np

Retrieving Fangraphs Data

In [84]:
fangraphs = pyb.pitching_stats(2021, qual = 0).sort_values('Name')
fangraphs

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
510,22343,2021,A.J. Alexy,TEX,23,3,1,-0.1,4.70,5,...,22.7,6,0.097,111.0,20,0.323,62,0.139,0.229,6.77
62,11467,2021,A.J. Cole,TOR,29,0,0,0.1,1.13,6,...,23.1,3,0.136,107.3,9,0.409,22,0.132,0.254,3.55
347,18655,2021,A.J. Minter,ATL,27,3,6,1.3,3.78,61,...,11.8,6,0.042,116.3,50,0.350,143,0.164,0.307,3.01
667,19343,2021,A.J. Puk,OAK,26,0,3,0.1,6.08,12,...,7.6,1,0.024,111.6,21,0.500,42,0.173,0.267,4.15
11,8350,2021,AJ Ramos,LAA,34,0,0,0.1,0.00,4,...,16.2,0,0.000,99.7,2,0.182,11,0.145,0.274,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,19159,2021,Zack Burdi,- - -,26,0,0,-0.2,5.40,7,...,21.4,5,0.143,112.5,16,0.457,35,0.136,0.271,6.60
874,14862,2021,Zack Godley,MIL,31,0,1,-0.3,16.20,2,...,12.8,1,0.111,111.2,5,0.556,9,0.136,0.261,9.40
402,1943,2021,Zack Greinke,HOU,37,11,6,1.3,4.16,30,...,10.6,35,0.065,113.2,191,0.354,539,0.176,0.267,4.35
183,15823,2021,Zack Littell,SFG,25,4,0,0.3,2.92,63,...,12.0,13,0.080,113.7,75,0.460,163,0.129,0.262,3.89


Changing .1 to 1/3 and .2 to 2/3 innings pitched

In [85]:
fangraphs['IP'] = fangraphs['IP'].astype(str)

In [86]:
for i in range(1, 909):
  if i % 26 == 0:
    continue
  else:
    
    if fangraphs['IP'][i][-2:] == ".1":
      fangraphs['IP'][i] = fangraphs['IP'][i].replace(".1", ".33")
    elif fangraphs['IP'][i][-2:] == ".2":
      fangraphs['IP'][i] = fangraphs['IP'][i].replace(".2", ".66")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [87]:
fangraphs['IP'] = fangraphs['IP'].astype(float)

Ordering By Last Name

In [89]:
fangraphs[['FName', 'LName']] = fangraphs['Name'].str.split(" ", n=1, expand = True)
fangraphs.at[61, 'LName'] = 'DeGrom'
fangraphs.at[765, 'LName'] = 'De Geus'
fangraphs = fangraphs.sort_values('LName')

In [90]:
fangraphs

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA,FName,LName
623,4994,2021,Fernando Abad,BAL,35,0,0,0.1,5.60,16,...,0.046,108.2,24,0.369,65,0.174,0.231,4.57,Fernando,Abad
730,20277,2021,Cory Abbott,CHC,25,0,0,-0.5,6.75,7,...,0.102,113.9,25,0.424,59,0.122,0.199,6.98,Cory,Abbott
575,17485,2021,Albert Abreu,NYY,25,2,0,-0.6,5.15,28,...,0.081,110.4,29,0.293,99,0.168,0.277,4.92,Albert,Abreu
634,16609,2021,Bryan Abreu,HOU,24,3,3,0.1,5.75,31,...,0.067,112.9,41,0.394,104,0.138,0.271,4.76,Bryan,Abreu
242,17092,2021,Domingo Acevedo,OAK,27,0,0,-0.1,3.27,10,...,0.065,104.0,11,0.355,31,0.109,0.253,4.06,Domingo,Acevedo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,19269,2021,T.J. Zeuch,TOR,25,0,2,-0.6,6.60,5,...,0.158,114.1,28,0.491,57,0.154,0.227,11.00,T.J.,Zeuch
531,14169,2021,Kyle Zimmer,KCR,29,4,1,-0.2,4.83,52,...,0.095,112.8,56,0.381,147,0.162,0.266,4.79,Kyle,Zimmer
558,20370,2021,Bruce Zimmermann,BAL,26,4,5,0.4,5.04,14,...,0.098,119.3,91,0.444,205,0.166,0.273,6.39,Bruce,Zimmermann
774,4505,2021,Jordan Zimmermann,MIL,35,0,0,-0.1,7.94,2,...,0.130,108.2,11,0.478,23,0.110,0.134,10.54,Jordan,Zimmermann


Calculating WHHIP

In [91]:
fangraphs['WHHIP'] = round((fangraphs['BB'] + fangraphs['HardHit']) / fangraphs['IP'],3)

WHHIP DataFrame

In [ ]:
df = pd.DataFrame({'FName': fangraphs['FName'],
                   'LName': fangraphs['LName'],
              'ERA': fangraphs['ERA'],
              'IP': fangraphs['IP'],
              'BB': fangraphs['BB'],
              'Hard Hits': fangraphs['HardHit'],
              'FIP': fangraphs['FIP'],
              'WHIP': fangraphs['WHIP'],
              'WHHIP': fangraphs['WHHIP']
              })
df = df.sort_values("LName")
df